In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np



In [ ]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
#apply label encoding to labels
df.labels = pd.factorize(df.labels)[0]
filepath = df['image'].values
labels = df['labels'].values
data_len = len(labels)
print(data_len)


In [ ]:
#split data to train and test
train_size = int(0.8*data_len)
print("train size: ",train_size )
print("test size: ",val_size )
train_label, test_label = labels[:train_size], labels[train_size:]
train_filepath, test_filepath = filepath[:train_size], filepath[train_size:]
print(train_label.shape)

In [ ]:
#load train images

train_image = []
dir1 = '../input/plant-pathology-2021-fgvc8/train_images/'
i = 0
for file in train_filepath:
    i+=1
    if i % 500 == 0:
        print("Train image#",i)
    img_clr = tf.io.read_file(dir1+file)
    img1 = tf.image.decode_image(img_clr,channels=1,dtype=tf.float32)
    # Resize images to a common size
    img = tf.image.resize(img1, size=(256,256))
    img = np.asarray(img).reshape(-1)
    #img = np.asarray(img).reshape([-1, 256*256])
    # Normalize
    img = img /255.
    #print(img)
    train_image.append(img)
    
print("shape of train_image",np.asarray(train_image).shape)

In [ ]:

# Training parameters.
learning_rate = 0.01
training_steps = 5000
batch_size = 512
display_step = 50

In [ ]:
# Use tf.data API to shuffle and batch TRAIN data.
train_data = tf.data.Dataset.from_tensor_slices((train_image,train_label))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [ ]:
# Training parameters.
learning_rate = 0.0001
training_steps = 1000

# dataset parameters.
num_classes = 12 # 0 to 3 classes
num_features = 256*256 # 28*28


In [ ]:
# apply LOGISTIC REGRESSION

# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")

# Bias of shape [10], the total number of classes.
b = tf.Variable(tf.zeros([num_classes]), name="bias")

# Logistic regression (Wx + b).
def logistic_regression(x):

    # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.matmul(x, W) + b)

# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):

    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)

    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# Accuracy metric.

def accuracy(y_pred, y_true):

    # Predicted class is the index of the highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

# Optimization process. 
def run_optimization(x, y):

# Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    # Compute gradients.
    gradients = g.gradient(loss, [W, b])
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W, b]))
    
# Run training for the given number of steps.
for epoch,(batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    if epoch % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("epoch: %i, loss: %f, accuracy: %f" % (epoch, loss, acc))


In [ ]:
#load test images

test_image=[]
dir1 = '../input/plant-pathology-2021-fgvc8/train_images/'
i = 0
for file in test_filepath:
    i+=1
    if i % 500 == 0:
        print("Test image#",i)
    img_clr = tf.io.read_file(dir1+file)
    img1 = tf.image.decode_image(img_clr,channels=1,dtype=tf.float32)
    # Resize images to a common size
    img = tf.image.resize(img1, size=(256,256))
    img = np.asarray(img).reshape(-1)
    #img = np.asarray(img).reshape([-1, 256*256])
    # Normalize
    img = img /255.
    #print(img)
    test_image.append(img)


In [ ]:
# Test model on validation set.

pred = logistic_regression(test_image)
print("Test Accuracy: %f" % accuracy(pred, test_label))